In [24]:
import warnings
warnings.filterwarnings('ignore')

In [25]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import pandas as pd
import sys
import os
from scipy import interpolate
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits
from scipy.optimize import curve_fit
import astropy.units as u
import astropy.constants as c

In [3]:
import libwwz

In [4]:
from astropy.timeseries import LombScargle

In [26]:
import stingray as sr
from stingray.events import EventList
from stingray.lightcurve import Lightcurve
from stingray import Lightcurve, Crossspectrum, sampledata,Powerspectrum,AveragedPowerspectrum
from stingray.simulator import simulator, models

In [5]:
help(libwwz)

Help on package libwwz:

NAME
    libwwz - Exposes the wwt function directly without import the wwz module.

PACKAGE CONTENTS
    plot_methods
    wwz

FILE
    /Users/baotong/opt/anaconda3/lib/python3.8/site-packages/libwwz/__init__.py




In [27]:
def get_hist(t, len_bin):
    ###将输入的time信息，按照len_bin的长度输出为lc
    t_test = t-t[0]
    a = [0 for i in range(int(t_test[-1] / len_bin) + 1)]
    for i in range(len(t_test)):
        a[int(t_test[i] / len_bin)] += 1
    a = np.array(a)
    return a

In [47]:
dt = 2  # seconds
long_dt = 2  # seconds
long_exposure = 160.  # seconds
long_times = np.arange(0, long_exposure, long_dt)  # seconds
# In count rate units here
long_signal = 100 * np.sin(2.*np.pi*long_times/10) + 1000

# Multiply by dt to get count units, then add Poisson noise
long_noisy = np.random.poisson(long_signal*long_dt)

In [48]:
long_lc = Lightcurve(long_times, long_noisy)

In [49]:
help(libwwz.wwt)

Help on function wwt in module libwwz.wwz:

wwt(timestamps: numpy.ndarray, magnitudes: numpy.ndarray, time_divisions: int, freq_params: list, decay_constant: float, method: str = 'linear', parallel: bool = True) -> numpy.ndarray
    The code is based on G. Foster's FORTRAN
    code as well as eaydin's python 2.7 code. The code is updated to use numpy methods and allow for float value tau.
    It returns an array with matrix of new evenly spaced timestamps, frequencies, wwz-power, amplitude, coefficients,
    and effective number. Specific equations can be found on Grant Foster's "WAVELETS FOR PERIOD ANALYSIS OF UNEVENLY
    SAMPLED TIME SERIES". Some of the equations are labeled in the code with corresponding numbers.
    
    :param timestamps: An array with corresponding times for the magnitude (payload).
    :param magnitudes: An array with payload values
    :param time_divisions: number of divisions for the new timestamps
    :param freq_params: A list containing parameters for ma

In [28]:
freq=np.arange(1/10.,1./0.5,1/500)

In [29]:
bin_len=0.1
path='/Users/baotong/Desktop/FRBtime/'
filename='FRB240114A_0306.txt'
time=np.loadtxt(path+filename)*86400
print(time)
T_exp = time[-1] - time[0]
# freq=np.arange(1/T_exp,0.5/bin_len,1/(5*T_exp))            
flux=get_hist(time,bin_len)
x=np.arange(bin_len/2.,(time[-1]-time[0])+bin_len/2.,bin_len)

[5.21641133e+09 5.21641133e+09 5.21641134e+09 5.21641138e+09
 5.21641138e+09 5.21641140e+09 5.21641142e+09 5.21641142e+09
 5.21641144e+09 5.21641144e+09 5.21641144e+09 5.21641145e+09
 5.21641145e+09 5.21641148e+09 5.21641150e+09 5.21641151e+09
 5.21641151e+09 5.21641152e+09 5.21641153e+09 5.21641154e+09
 5.21641154e+09 5.21641155e+09 5.21641155e+09 5.21641155e+09
 5.21641157e+09 5.21641158e+09 5.21641158e+09 5.21641160e+09
 5.21641161e+09 5.21641161e+09 5.21641164e+09 5.21641165e+09
 5.21641166e+09 5.21641168e+09 5.21641168e+09 5.21641169e+09
 5.21641170e+09 5.21641170e+09 5.21641171e+09 5.21641172e+09
 5.21641172e+09 5.21641173e+09 5.21641174e+09 5.21641174e+09
 5.21641174e+09 5.21641174e+09 5.21641176e+09 5.21641176e+09
 5.21641178e+09 5.21641178e+09 5.21641178e+09 5.21641180e+09
 5.21641181e+09 5.21641181e+09 5.21641181e+09 5.21641183e+09
 5.21641183e+09 5.21641184e+09 5.21641187e+09 5.21641187e+09
 5.21641187e+09 5.21641188e+09 5.21641189e+09 5.21641189e+09
 5.21641189e+09 5.216411

In [30]:
[Tau, Freq, WWZ, AMP, COEF, NEFF]=libwwz.wwt(timestamps=x,magnitudes=flux,time_divisions=10,freq_params=[freq[0],freq[-1],freq[-1]-freq[-2],False],decay_constant=1e-3)

*** Starting Weighted Wavelet Z-transform ***

Pseudo sample frequency (median) is  10.0
largest tau window is  792.867
6.69 seconds has passed to complete Weighted Wavelet Z-transform 



In [32]:
%matplotlib widget      
fig, ax = plt.subplots()
ax.set_title('test title')
libwwz.plot_methods.linear_plotter(ax,Tau,Freq,WWZ)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

KeyError: 'default'

In [1]:
path='/Users/baotong/Desktop/CDFS/'
file1='145108.76+270926.9_lccorr_1000_soft.fits'
file2='145108.76+270926.9_lccorr_1000_soft_2nd.fits'

In [ ]:
fits.open(path+file1)[1].data